In [1]:
import glob
import json
import os
import shutil

In [9]:
def process_iocage_plugin_2(manifest):
    manifest = os.path.abspath(manifest)

    assert os.path.exists(manifest), FileNotFoundError(manifest)

    path, basename = os.path.split(manifest)
    iocage_name, ext = os.path.splitext(basename)
    dirname = os.path.basename(path)
    assert iocage_name == dirname
    assert ext == ".json"
    # Open file for reading.
    with open(manifest, "r") as fid:
        # Load the json.
        try:
            plugin_json = json.load(fid)
        except json.JSONDecodeError:
            raise Exception(manifest)
        except:
            raise
    #
    # Plugin Tests.
    plugin_json["release"] = "12.0-RELEASE"
    assert plugin_json["release"] == "12.0-RELEASE"
    plugin_json["plugin_schema"] = "2"
    assert plugin_json["plugin_schema"] == "2"
    
    if "properties" not in plugin_json:
        plugin_json["properties"] = {
            "allow_raw_sockets": "1",
            "boot": "on",
            "bpf": "yes",
            "dhcp": "on",
            "vnet": "on"
        }

    # Open file for writing.
    with open(manifest, "w") as fid:
        # Pretty print, indent, and sort keys.
        json.dump(plugin_json, fid, indent=4, sort_keys=True)

    shutil.copy(manifest, os.path.dirname(path))
    return (
        iocage_name,
        dict(
            {
                "MANIFEST": basename,
                "name": iocage_name,
                "official": False,
                "description": "TODO:",
                "icon": "",
            }
        ),
    )


In [11]:
INDEX = dict()
manifests = glob.glob("*/*.json")
for manifest in manifests:
    if "template" in manifest:
        continue
    manifest = os.path.abspath(manifest)
    key, data = process_iocage_plugin_2(manifest)
    INDEX[key] = data
# Open the index file.
with open("INDEX", "w") as fid:
    # Pretty print, indent, and sort keys.
    json.dump(INDEX, fid, indent=4, sort_keys=True)
